## The process for obtaining token-level rationales consists of three steps.

In [1]:
import sys

sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

## Step 1: Model Preparing
### Load the pretrained language model and the dataset

In [2]:
import numpy as np
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = "ernie-1.0"

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

In [3]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

### Train the model